In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# The path to our CSV files
2011_csv_path = ".data/Medicare_Provider_Charge_Inpatient_DRG100_FY2011.csv"
# 2012_csv_path = ".data/Medicare_Provider_Charge_Inpatient_DRG100_FY2012.csv"
# 2013_csv_path = ".data/Medicare_Provider_Charge_Inpatient_DRG100_FY2013.csv"
# 2014_csv_path = ".data/Medicare_Provider_Charge_Inpatient_DRG100_FY2014.csv"
# 2015_csv_path = ".data/Medicare_Provider_Charge_Inpatient_DRG100_FY2015.csv"

# Read our data into pandas
2011_df = pd.read_csv(2011_csv_path)
# 2012_df = pd.read_csv(2012_csv_path)
# 2013_df = pd.read_csv(2013_csv_path)
# 2014_df = pd.read_csv(2014_csv_path)
# 2015_df = pd.read_csv(2015_csv_path)
df= pd.merge(2011_df,2012_df,2013_df,2014_df,2015_df)